In [1]:
# Standard python helper libraries.
import os, sys, re, json, time, wget, csv, string, time, random
import itertools, collections

# NumPy
import numpy as np

# NLTK for NLP utils
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from keras.models import Sequential
from keras.layers import GaussianNoise, Dropout, Dense, Embedding, MaxPool1D, GlobalMaxPool1D, Conv1D
from keras.optimizers import Adam

from pymagnitude import *

[nltk_data] Downloading package punkt to /home/renzeer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/renzeer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [2]:
med_vectors = Magnitude("data/wikipedia-pubmed-and-PMC-w2v.magnitude", pad_to_length=30)

In [16]:
ehr_notes = []
with open('data/ehr_samples.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:     
        if int(row['SpecialtyID']) in [39, 6, 16, 37, 11, 12, 29, 26, 7, 21, 19, 10, 2, 18]:
            continue
        elif int(row['SpecialtyID']) != 4:
            ehr_notes.append(['Other', row['Note']])
        else:
            ehr_notes.append([row['Specialty'], row['Note']])

print(ehr_notes[0])
print(ehr_notes[2])

['Other', 'PAST MEDICAL HISTORY:, He has difficulty climbing stairs, difficulty with airline seats, tying shoes, used to public seating, and lifting objects off the floor.  He exercises three times a week at home and does cardio.  He has difficulty walking two blocks or five flights of stairs.  Difficulty with snoring.  He has muscle and joint pains including knee pain, back pain, foot and ankle pain, and swelling.  He has gastroesophageal reflux disease.,PAST SURGICAL HISTORY:, Includes reconstructive surgery on his right hand 13 years ago.  ,SOCIAL HISTORY:, He is currently single.  He has about ten drinks a year.  He had smoked significantly up until several months ago.  He now smokes less than three cigarettes a day.,FAMILY HISTORY:, Heart disease in both grandfathers, grandmother with stroke, and a grandmother with diabetes.  Denies obesity and hypertension in other family members.,CURRENT MEDICATIONS:, None.,ALLERGIES:,  He is allergic to Penicillin.,MISCELLANEOUS/EATING HISTORY:

In [17]:
ehr_sentences = []
for record in ehr_notes:
    sent_text = nltk.sent_tokenize(record[1])
    for sent in sent_text:
        tokens = word_tokenize(sent)

        # convert to lower case
        tokens = [w.lower() for w in tokens]

        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]

        # filter out stop words
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]

#         # stem words
#         porter = PorterStemmer()
#         tokens = [porter.stem(word) for word in tokens]

        # remove blanks
        tokens = [w for w in tokens if w != '']

        ehr_sentences.append([record[0], ' '.join(tokens)])

random.Random(4).shuffle(ehr_sentences)

print(ehr_sentences[0])
print(ehr_sentences[1])

['Cardiovascular', 'return clinic 4 weeks5']
['Other', 'biceps tendon nonsubluxable']


## Defining our Training and Test Data

Before we can start building our neural networks, we first have to define our datasets. Specifically, we have to break up our EHR data so that we have records that we can train on and records that are exclusively used to test on. Maintaining a separate set for testing ensures we avoid overfitting our data.

We will use some built-in functions provided by Magnitude that helps encode our classes/categories. We then partition our data into our train and test sets. For each set we have both data and labels. Initially, we will be making these partitions small to make iterating through model development much quicker. However, once the models are developed, we will expand our datasets to include all of our data. To ensure we defined our data correctly, we can print a few lines from the two sets.

In [18]:
len(ehr_sentences)

70941

In [19]:
add_intent, intent_to_int, int_to_intent = MagnitudeUtils.class_encoding()

x_train = [ehr_sent[1].split(' ') for ehr_sent in ehr_sentences[:60000]]
x_test = [ehr_sent[1].split(' ') for ehr_sent in ehr_sentences[60001:]]

y_train = [add_intent(ehr_sent[0]) for ehr_sent in ehr_sentences[:60000]]
y_test = [add_intent(ehr_sent[0]) for ehr_sent in ehr_sentences[60001:]]

y_train = list(np.array(y_train).reshape(len(y_train)))
y_test = list(np.array(y_test).reshape(len(y_test)))

num_training = len(x_train)
num_test = len(x_test)
num_outputs = int(max(max(y_train), max(y_test))) + 1

print(int_to_intent(0))

print("First line of train/test data:")
print("\t", x_train[0])
print("\t", y_train[0], int_to_intent(y_train[0]))
print("\t", x_test[0])
print("\t", y_test[0], int_to_intent(y_test[0]))
print("Second line of train/test data:")
print("\t", x_train[1])
print("\t", y_train[1], int_to_intent(y_train[1]))
print("\t", x_test[1])
print("\t", y_test[1], int_to_intent(y_test[1]))

Cardiovascular
First line of train/test data:
	 ['return', 'clinic', '4', 'weeks5']
	 0 Cardiovascular
	 ['smokes', 'one', 'pack', 'per', 'day']
	 0 Cardiovascular
Second line of train/test data:
	 ['biceps', 'tendon', 'nonsubluxable']
	 1 Other
	 ['denies', 'illicit', 'drug', 'use', 'family', 'history', 'parents', 'died', 'myocardial', 'infarctions']
	 1 Other


In [20]:
MAX_WORDS = 30 # The maximum number of words the sequence model will consider
STD_DEV = 0.01 # Deviation of noise for Gaussian Noise applied to the embeddings
DROPOUT_RATIO = .5 # The ratio to dropout
BATCH_SIZE = 100 # The number of examples per train/validation step
EPOCHS = 100 # The number of times to repeat through all of the training data
LEARNING_RATE = .01 # The learning rate for the optimizer
NUM_FILTERS = 128

model = Sequential()
model.add(GaussianNoise(STD_DEV, input_shape=(MAX_WORDS, med_vectors.dim)))
model.add(Conv1D(NUM_FILTERS, 7, activation='relu', padding='same'))
model.add(MaxPool1D(2))
model.add(Conv1D(NUM_FILTERS, 7, activation='relu', padding='same'))
model.add(GlobalMaxPool1D())
model.add(Dropout(DROPOUT_RATIO))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_outputs, activation='softmax'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_1 (GaussianNo (None, 30, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 128)           179328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 15, 128)           114816    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
__________

In [21]:
training_batches = MagnitudeUtils.batchify(x_train, y_train, BATCH_SIZE) # Split the training data into batches
num_batches_per_epoch_train = int(np.ceil(num_training/float(BATCH_SIZE)))
test_batches = MagnitudeUtils.batchify(x_test, y_test, BATCH_SIZE)  # Split the test data into batches
num_batches_per_epoch_test = int(np.ceil(num_test/float(BATCH_SIZE)))


# Generates batches of the transformed training data
train_batch_generator = (
  (
    med_vectors.query(x_train_batch), # Magnitude will handle converting the 2D array of text into the 3D word vector representations!
    MagnitudeUtils.to_categorical(y_train_batch, num_outputs) # Magnitude will handle converting the class labels into one-hot encodings!
  ) for x_train_batch, y_train_batch in training_batches
)

# Generates batches of the transformed test data
test_batch_generator = (
  (
    med_vectors.query(x_test_batch), # Magnitude will handle converting the 2D array of text into the 3D word vector representations!
    MagnitudeUtils.to_categorical(y_test_batch, num_outputs) # Magnitude will handle converting the class labels into one-hot encodings!
  ) for x_test_batch, y_test_batch in test_batches
)

# Start training
from keras.utils import np_utils
model.fit_generator(
    generator = train_batch_generator,
    steps_per_epoch = num_batches_per_epoch_train,
    validation_data = test_batch_generator,
    validation_steps = num_batches_per_epoch_test,
    epochs = EPOCHS,
)

Epoch 1/100
600/600 [==============================] - 325s 542ms/step - loss: 0.3283 - acc: 0.8698 - val_loss: 0.3012 - val_acc: 0.8827
Epoch 2/100
600/600 [==============================] - 38s 63ms/step - loss: 0.2771 - acc: 0.8909 - val_loss: 0.2823 - val_acc: 0.8857
Epoch 3/100
600/600 [==============================] - 38s 63ms/step - loss: 0.2400 - acc: 0.9044 - val_loss: 0.2873 - val_acc: 0.8843
Epoch 4/100
600/600 [==============================] - 38s 64ms/step - loss: 0.2012 - acc: 0.9173 - val_loss: 0.3143 - val_acc: 0.8782
Epoch 5/100
600/600 [==============================] - 35s 59ms/step - loss: 0.1766 - acc: 0.9253 - val_loss: 0.3276 - val_acc: 0.8676
Epoch 6/100
600/600 [==============================] - 36s 59ms/step - loss: 0.1593 - acc: 0.9321 - val_loss: 0.3730 - val_acc: 0.8505
Epoch 7/100
600/600 [==============================] - 37s 62ms/step - loss: 0.1441 - acc: 0.9387 - val_loss: 0.4094 - val_acc: 0.8604
Epoch 8/100
600/600 [==============================] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



600/600 [==============================] - 38s 63ms/step - loss: 0.0807 - acc: 0.9595 - val_loss: 0.6350 - val_acc: 0.8590
Epoch 27/100
600/600 [==============================] - 38s 63ms/step - loss: 0.0782 - acc: 0.9600 - val_loss: 0.6436 - val_acc: 0.8559
Epoch 28/100
600/600 [==============================] - 38s 64ms/step - loss: 0.0758 - acc: 0.9608 - val_loss: 0.6635 - val_acc: 0.8489
Epoch 29/100
600/600 [==============================] - 39s 64ms/step - loss: 0.0779 - acc: 0.9604 - val_loss: 0.6573 - val_acc: 0.8562
Epoch 30/100
600/600 [==============================] - 38s 63ms/step - loss: 0.0754 - acc: 0.9605 - val_loss: 0.7012 - val_acc: 0.8620
Epoch 31/100
469/600 [======================>.......] - ETA: 7s - loss: 0.0726 - acc: 0.9625

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



600/600 [==============================] - 37s 62ms/step - loss: 0.0737 - acc: 0.9617 - val_loss: 0.7328 - val_acc: 0.8622
Epoch 34/100
600/600 [==============================] - 37s 62ms/step - loss: 0.0720 - acc: 0.9613 - val_loss: 0.7432 - val_acc: 0.8581
Epoch 35/100
600/600 [==============================] - 35s 59ms/step - loss: 0.0724 - acc: 0.9617 - val_loss: 0.7661 - val_acc: 0.8612
Epoch 36/100
600/600 [==============================] - 36s 61ms/step - loss: 0.0715 - acc: 0.9629 - val_loss: 0.7335 - val_acc: 0.8649
Epoch 37/100
600/600 [==============================] - 40s 66ms/step - loss: 0.0694 - acc: 0.9625 - val_loss: 0.7892 - val_acc: 0.8676
Epoch 38/100
600/600 [==============================] - 38s 63ms/step - loss: 0.0701 - acc: 0.9626 - val_loss: 0.7734 - val_acc: 0.8644
Epoch 39/100
600/600 [==============================] - 38s 63ms/step - loss: 0.0674 - acc: 0.9634 - val_loss: 0.8142 - val_acc: 0.8658
Epoch 40/100
600/600 [==============================] - 38s 6

600/600 [==============================] - 39s 66ms/step - loss: 0.0571 - acc: 0.9659 - val_loss: 1.1653 - val_acc: 0.8687
Epoch 94/100
600/600 [==============================] - 38s 63ms/step - loss: 0.0564 - acc: 0.9669 - val_loss: 1.1857 - val_acc: 0.8656
Epoch 95/100
600/600 [==============================] - 36s 60ms/step - loss: 0.0583 - acc: 0.9656 - val_loss: 1.1079 - val_acc: 0.8665
Epoch 96/100
600/600 [==============================] - 38s 63ms/step - loss: 0.0577 - acc: 0.9663 - val_loss: 1.1810 - val_acc: 0.8698
Epoch 97/100
600/600 [==============================] - 38s 64ms/step - loss: 0.0575 - acc: 0.9665 - val_loss: 1.1722 - val_acc: 0.8677
Epoch 98/100
600/600 [==============================] - 38s 64ms/step - loss: 0.0564 - acc: 0.9668 - val_loss: 1.1345 - val_acc: 0.8665
Epoch 99/100
600/600 [==============================] - 39s 64ms/step - loss: 0.0554 - acc: 0.9666 - val_loss: 1.1799 - val_acc: 0.8694
Epoch 100/100
600/600 [==============================] - 39s 

In [22]:
print("Results after training for %d epochs:" % (EPOCHS,))

train_metrics = model.evaluate_generator(
    generator = train_batch_generator,
    steps = num_batches_per_epoch_train,
)

print("loss: %.4f - categorical_accuracy: %.4f" % tuple(train_metrics))

val_metrics = model.evaluate_generator(
    generator = test_batch_generator,
    steps = num_batches_per_epoch_test,
)

print("val_loss: %.4f - val_categorical_accuracy: %.4f" % tuple(val_metrics))

Results after training for 100 epochs:
loss: 0.0506 - categorical_accuracy: 0.9680
val_loss: 1.1566 - val_categorical_accuracy: 0.8673
